# Visibility Simulator Examples

Although ``hera_sim`` is primarily aimed at simulating instrumental effects _on top_ of existing visibility simulations, it also provides a neat interface to several visibility simulators.

It does so through the base class ``VisibilitySimulator``, which sets a standard API from which all simulators must derive.

## VisCPU

In [ ]:
from hera_sim.visibilities import VisCPU, HealVis#, VisGPU#, PRISim
from hera_sim import io
import numpy as np
import healpy
import matplotlib.pyplot as plt
import itertools
import copy
from pyuvsim.analyticbeam import AnalyticBeam
from astropy.units import sday

import healvis as HEALVIS

%matplotlib inline
%load_ext autoreload
%autoreload 2

#### HEALVIS GIVES LONG WARNINGS
#import warnings
#warnings.filterwarnings("ignore")

[autoreload of _frozen_importlib failed: Traceback (most recent call last):
  File "/home/jackson/anaconda3/envs/py3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, reload, self.old_objects)
  File "/home/jackson/anaconda3/envs/py3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 434, in superreload
    module = reload(module)
  File "/home/jackson/anaconda3/envs/py3/lib/python3.6/imp.py", line 315, in reload
    return importlib.reload(module)
  File "/home/jackson/anaconda3/envs/py3/lib/python3.6/importlib/__init__.py", line 165, in reload
    spec = module.__spec__ = _bootstrap._find_spec(name, pkgpath, target)
AttributeError: module 'importlib._bootstrap' has no attribute '_find_spec'
]
[autoreload of _frozen_importlib_external failed: Traceback (most recent call last):
  File "/home/jackson/anaconda3/envs/py3/lib/python3.6/site-packages/IPython/extensions/autoreload.py", line 245, in check
    superreload(m, r

In [ ]:
### np.unique can reorder elements
def unique(arr):
    arr = np.array(arr)
    indices = np.unique(arr, return_index=True)[1]
    return np.array([arr.flatten()[i] for i in sorted(indices)])

def plot_blts(simulator):
    fig, ax = plt.subplots(2, 1, sharex=True, 
                           subplot_kw={"ylabel":"|Visibility| [Jy]"}, figsize=(16,8))
    
    fig.suptitle(type(simulator).__name__, fontweight='bold')
    
    lw = 2
    
    for i in simulator.uvdata.antenna_numbers:
        auto_ind = simulator.uvdata.antpair2ind((i,i))
        #ax[0].plot(unique(simulator.uvdata.lst_array)*12/np.pi, 
        #   simulator.uvdata.data_array[auto_ind, 0, 0, 0], label="ant_{}".format(i))
        
        ax[0].plot(np.real(simulator.uvdata.data_array[auto_ind, 0, 0, 0]), label="ant_{} real".format(i), linewidth=lw)
        ax[0].plot(np.imag(simulator.uvdata.data_array[auto_ind, 0, 0, 0]), linestyle="--", label="ant_{} imag".format(i), linewidth=lw)

        for j in simulator.uvdata.antenna_numbers[(i+1):]:
            cross_ind = simulator.uvdata.antpair2ind((i,j))
            #ax[1].plot(unique(simulator.uvdata.lst_array)*12/np.pi, simulator.uvdata.data_array[cross_ind, 0, 0, 0], label="bl({},{})".format(i,j))
            ax[1].plot(np.real(simulator.uvdata.data_array[cross_ind, 0, 0, 0]), label="bl({},{}) real".format(i,j), linewidth=lw)
            ax[1].plot(np.imag(simulator.uvdata.data_array[cross_ind, 0, 0, 0]), linestyle="--", label="bl({},{}) imag".format(i,j), linewidth=lw)
            
    ax[0].legend()
    ax[1].legend()
    ax[0].set_title("Autocorrelations")
    ax[1].set_title("Cross-Correlations")
    ax[1].set_xlabel("Observation Number")

def compare_blts(simulator1, simulator2):
    fig, ax = plt.subplots(3, 1, sharex=True, 
                           subplot_kw={"ylabel":"|Visibility| [Jy]"}, figsize=(16, 12))
    
    name1 = type(simulator1).__name__
    name2 = type(simulator2).__name__
    
    fig.suptitle(name1 + " vs " + name2, fontweight='bold')
    
    lw=3

    for i in simulator1.uvdata.antenna_numbers:
        auto_ind = simulator1.uvdata.antpair2ind((i,i))
        auto1 = np.real(simulator1.uvdata.data_array[auto_ind, 0, 0, 0])
        auto2 = np.real(simulator2.uvdata.data_array[auto_ind, 0, 0, 0])
                        
        ax[0].plot(auto1, label=("ant_{} " + name1).format(i), linewidth=lw)
        ax[0].plot(auto2, linestyle="--", label=("ant_{} "+name2).format(i), linewidth=lw)

        for j in simulator1.uvdata.antenna_numbers[(i+1):]:
            cross_ind = simulator1.uvdata.antpair2ind((i,j))
            cross1 = np.real(simulator1.uvdata.data_array[cross_ind, 0, 0, 0])
            cross2 = np.real(simulator2.uvdata.data_array[cross_ind, 0, 0, 0])
                        
            ax[1].plot(cross1, label=("bl({},{}) " + name1).format(i,j), linewidth=lw)
            ax[1].plot(cross2, linestyle="--", label=("bl({},{}) " + name2).format(i,j), linewidth=lw)
            ax[2].plot(cross2 - cross1, label=("bl({},{}) " + "Difference").format(i,j), linewidth=lw, c="k")
    
    [a.legend() for a in ax]
    ax[0].set_title("Autocorrelations")
    ax[1].set_title("Cross-Correlations")
    ax[2].set_title(name2 + " Cross-Correlation minus " + name1 + " Cross-Correlation")
    ax[1].set_xlabel("Observation Number")

``VisCPU`` natively deals with diffuse emission, and the ``VisibilitySimulator`` uses ``healpix`` maps to represent such emission:

In [ ]:
def uniform_sky(nside_base=5, scale=1, nfreqs=5):
    NSIDE = 2**nside_base
    NPIX = 12 * NSIDE**2 # this is the number of pixels a healpix map has for  given NSIDE

    # Create a sky of purely ones, multiply by "scale"
    return scale * np.ones((nfreqs, NPIX))/NPIX

Other than the sky model (we'll revisit this soon), the ``VisibilitySimulator`` requires a ``UVData`` object to specify the observation parameters, as well as a list of ``UVBeam``s to specify the beams for each antenna. This makes it fully consistent with the interface provided by ``pyuvsim``, and many of the methods found in ``pyuvsim`` for creating ``UVData`` objects and ``UVBeam`` objects from YAML specifications can be used here. Indeed, a dictionary of so-called ``obsparams``, or a file-name for an appropriate YAML file, can be passed in lieu of the ``UVData`` and ``UVBeam``. 

For simplicity, let us simply use the native ``empty_uvdata`` function instead:

In [ ]:
def myuvdata(ntimes=48, ntel=1, **kwargs):
    
    # Picking antenna postions in a deterministic way
    ants = {}
    for i in range(ntel):
        ants.update([(i, (-(i % 3), i, i % 4))])
      
    return io.empty_uvdata(
        nfreq = 5,
        integration_time = sday.to('s')/ntimes,
        ntimes= ntimes,
        ants = ants,
        **kwargs
    )

In [ ]:
uvdata = myuvdata(ntel=2)
print(uvdata.telescope_location)
print(uvdata.telescope_location_lat_lon_alt)
print(uvdata.telescope_location_lat_lon_alt_degrees)
print(uvdata.ant1)

In [ ]:
uvdata = myuvdata(ntel=2)
freqs = unique(uvdata.freq_array)

simulator = VisCPU(
    uvdata=uvdata,
    sky_freqs=freqs,
    sky_intensity=uniform_sky(5),
    real_dtype=np.float64,
    complex_dtype=np.complex128, 
)

In [ ]:
vis = simulator.simulate()
plot_blts(simulator)

Test whether increasing the number of pixels changes much

In [ ]:
uvdata = myuvdata(ntel=2)
simulator = VisCPU(
    uvdata=uvdata,
    sky_freqs=freqs,
    sky_intensity=uniform_sky(6),
    real_dtype=np.float64,
    complex_dtype=np.complex128, 
)

In [ ]:
vis = simulator.simulate()
plot_blts(simulator)

### Point Sources

In [ ]:
uvdata1 = myuvdata(ntel=3, start_jd=2458119.5)
freqs = unique(uvdata1.freq_array)

point_source_pos1 = np.array([(0, uvdata1.telescope_lat_lon_alt[0])]) # ra, dec)
point_source_flx1 = np.array([[1.0]*len(freqs)]).T

uvdata2 = myuvdata(ntel=3, start_jd=2458119.0)
point_source_pos1 = np.array([(0, uvdata2.telescope_lat_lon_alt[0])]) # ra, dec)
point_source_flx1 = np.array([[1.0]*len(freqs)]).T


In [ ]:
simulator = VisCPU(
    uvdata = myuvdata(ntel=3, start_jd = 2458119.5,),
    point_source_pos=point_source_pos1,
    point_source_flux = point_source_flx1,
    sky_freqs = freqs,
)

simulator_jd = VisCPU(
    uvdata = myuvdata(ntel=3, start_jd = 2458119.0,),
    point_source_pos=point_source_pos1,
    point_source_flux = point_source_flx1,
    sky_freqs = freqs,
)

In [ ]:
vis = simulator.simulate()
plot_blts(simulator)

vis_jd = simulator_jd.simulate()
plot_blts(simulator_jd)

In [ ]:
print "Eq2Top for .5: ", simulator.get_eq2tops()[:, 0, 0]
print "Eq2Top for .0: ", simulator_jd.get_eq2tops()[:, 0, 0]


In [ ]:
print np.unique(simulator.uvdata.lst_array)
print np.unique(simulator_jd.uvdata.lst_array)

In [ ]:
simulator.uvdata.lst_array

In [ ]:
simulator.get_eq2tops()

In [ ]:
point_source_pos = np.array([ # ra, dec
    (0, myuvdata().telescope_lat_lon_alt[0]),
    (0.1, myuvdata().telescope_lat_lon_alt[0])
]) 
point_source_flx = np.array([[1.0, 1.0]]*len(freqs))

In [ ]:
simulator = VisCPU(
    uvdata = myuvdata(),
    point_source_pos=point_source_pos,
    point_source_flux = point_source_flx,
    sky_freqs = freqs,
)

In [ ]:
vis = simulator.simulate()
print np.max(np.abs(vis-simulator.uvdata.data_array))


plot_blts(simulator)

In this, the beam was not set, and therefore the beam was assumed to be unity everywhere above the horizon. The beam can be set as a series of healpix maps, one for each antenna. 
An extra list of ``beam_ids`` specifies which of the beams each antenna uses. So, we can create something like a Gaussian beam:

In [ ]:
simulator = VisCPU(
    uvdata = myuvdata(),
    sky_intensity = uniform_sky(6),
    sky_freqs = freqs,
    beams = [AnalyticBeam("gaussian", diameter=14)]
)

In [ ]:
vis = simulator.simulate()
plot_blts(simulator)

## HealVis

We could also have used healvis for this:

In [ ]:
simulator = HealVis(
    uvdata = myuvdata(ntel=3),
    sky_freqs = freqs,
    sky_intensity = 100*uniform_sky(6),
)

In [ ]:
vis = simulator.simulate()

In [ ]:
plot_blts(simulator)
fig = plt.gcf()

In [ ]:
simulator = HealVis(
    uvdata = myuvdata(ntimes=12, ntel=3),
    point_source_pos=point_source_pos,
    point_source_flux = point_source_flux,
    sky_freqs = freqs,
)

In [ ]:
vis = simulator.simulate()
plot_blts(simulator)

## Comparing VisCPU & HealVis

### Airy Beams

In [ ]:
uvdata = myuvdata(ntel=1, ntimes=48)
uvdata1 = myuvdata(ntel=1, ntimes=48)
freqs = unique(uvdata.freq_array)

point_source_pos = np.array([[0, uvdata.telescope_location_lat_lon_alt[0]]])
point_source_flux = np.array([[1.0]] * len(freqs))

viscpu_sim = VisCPU(
    uvdata=uvdata,
    sky_freqs=freqs,
    point_source_pos=point_source_pos,
    point_source_flux=point_source_flux,
    nside=2**4,
    beams = [AnalyticBeam("airy", diameter=2)],
    real_dtype=np.float64,
    complex_dtype=np.complex128,
)

healvis_sim = HealVis(
    uvdata=uvdata1,
    sky_freqs=freqs,
    point_source_pos=point_source_pos,
    point_source_flux=point_source_flux,
    beams = [AnalyticBeam("airy", diameter=2)],
    nside=2 ** 4
)

viscpu = viscpu_sim.simulate()
healvis = healvis_sim.simulate()

#plot_blts(viscpu_sim)
#plot_blts(healvis_sim)

compare_blts(viscpu_sim, healvis_sim)

In [ ]:
uvdata = myuvdata()
uvdata1 = myuvdata()
freqs = unique(uvdata.freq_array)

viscpu_sim = VisCPU(
    uvdata=uvdata1,
    sky_freqs=freqs,
    sky_intensity=uniform_sky(6),
    nside=2**4,
    real_dtype=np.float64,
    complex_dtype=np.complex128, 
)

healvis_sim = HealVis(
    uvdata=uvdata,
    sky_freqs=freqs,
    sky_intensity=uniform_sky(6),
    nside=2 ** 4
)

viscpu = viscpu_sim.simulate()
healvis = healvis_sim.simulate()

print "MAX ABSOLUTE DIFF", np.max(np.abs(viscpu-healvis))

plot_blts(viscpu_sim)
plot_blts(healvis_sim)
plt.figure()

### 2 Telescope Uniform Sky

In [ ]:
uvdata = myuvdata(ntel=2)
uvdata1 = myuvdata(ntel=2)

freqs = np.unique(uvdata.freq_array)

viscpu_sim = VisCPU(
    uvdata=myuvdata(ntel=3),
    sky_freqs=freqs,
    sky_intensity=uniform_sky(6),
    nside=2**4,
    real_dtype=np.float64,
    complex_dtype=np.complex128, 
)

healvis_sim = HealVis(
    uvdata=myuvdata(ntel=3),
    sky_freqs=freqs,
    sky_intensity=uniform_sky(6),
    nside=2 ** 4
)

viscpu = viscpu_sim.simulate()
healvis = healvis_sim.simulate()

viscpu = viscpu_sim.uvdata.data_array
healvis = healvis_sim.uvdata.data_array

print "MAX ABSOLUTE DIFF", np.max(np.abs(np.real(viscpu)-np.real(healvis)))

plot_blts(viscpu_sim)
plot_blts(healvis_sim)
plt.figure()


print unique(healvis_sim.uvdata.lst_array)*12/np.pi
print np.unique(healvis_sim.uvdata.lst_array)*12/np.pi

In [ ]:
print healvis_sim.uvdata.antpair2ind((0,0))
print healvis_sim.uvdata.antpair2ind((0,1))
print healvis_sim.uvdata.antpair2ind((1,0))
print healvis_sim.uvdata.antpair2ind((1,1))
print healvis_sim.uvdata.data_array.shape

### ??? 2 Telescope Hemispherical Source ???

In [ ]:
sky_intensity = uniform_sky(6)
sky_intensity[:,:sky_intensity.shape[1]/2] = 0.0
print sky_intensity

uvdata = myuvdata(ntel=2)
uvdata1 = myuvdata(ntel=2)

freqs = np.unique(uvdata.freq_array)

viscpu_sim = VisCPU(
    uvdata=uvdata,
    sky_freqs=freqs,
    sky_intensity=sky_intensity,
    nside=2**4,
    real_dtype=np.float64,
    complex_dtype=np.complex128, 
)

healvis_sim = HealVis(
    uvdata=uvdata1,
    sky_freqs=freqs,
    sky_intensity=sky_intensity,
    nside=2 ** 4
)

viscpu = viscpu_sim.simulate()
healvis = healvis_sim.simulate()

viscpu = viscpu_sim.uvdata.data_array
healvis = healvis_sim.uvdata.data_array

print "MAX ABSOLUTE DIFF", np.max(np.abs(np.real(viscpu)-np.real(healvis)))

plot_blts(viscpu_sim)
plot_blts(healvis_sim)
plt.figure()

### 2 Telescope Point Source at Zenith

In [ ]:
def convert_point_sources_to_healpix(point_source_pos, point_source_flux, nside=2**5):
    """
    Convert a set of point sources to an approximate diffuse healpix model.

    The healpix map returned is in RING scheme.

    Returns:
        2D array, shape=[NFREQ, NPIX]: the healpix diffuse model.
    """

    hmap = np.zeros((len(point_source_flux), healpy.nside2npix(nside)))

    # Get which pixel every point source lies in.
    pix = healpy.ang2pix(nside, np.pi/2 - point_source_pos[:, 1], point_source_pos[:, 0])

    hmap[:, pix] += point_source_flux / healpy.nside2pixarea(nside)

    return hmap


def convert_healpix_to_point_sources(hmap):
    """
    Convert a healpix map to a set of point sources located at the centre
    of each pixel.

    Args:
        hmap (2D array, shape[NFREQ, NPIX]):
            The healpix map.
    Returns:
        2D array: the point sources
    """
    nside = healpy.get_nside(hmap[0])
    ra, dec = healpy.pix2ang(nside, np.arange(len(hmap[0])), lonlat=True)
    flux = hmap * healpy.nside2pixarea(nside)
    return np.array([ra*np.pi/180, dec*np.pi/180]).T, flux

In [ ]:
uvdata = myuvdata(ntel=2, ntimes=10)
uvdata1 = myuvdata(ntel=2, ntimes=10)
freqs = np.unique(uvdata.freq_array)
nside = 2**4


point_source_pos = np.array([[0, uvdata.telescope_location_lat_lon_alt[0]+np.pi/2]])
point_source_flux = np.array([[1.0]] * len(freqs))

hmap = convert_point_sources_to_healpix(point_source_pos, point_source_flux, nside=nside)
point_source_pos1, point_source_flux1 = convert_healpix_to_point_sources(hmap)


viscpu_sim = VisCPU(
    uvdata=uvdata1,
    sky_freqs=freqs,
    point_source_pos=point_source_pos1,
    point_source_flux=point_source_flux1,
    nside=nside,
    real_dtype=np.float64,
    complex_dtype=np.complex128,
)

healvis_sim = HealVis(
    uvdata=uvdata,
    sky_freqs=freqs,
    point_source_pos=point_source_pos,
    point_source_flux=point_source_flux,
    nside=nside
)

viscpu_sim.simulate()
healvis_sim.simulate()

viscpu = viscpu_sim.uvdata.data_array
healvis = healvis_sim.uvdata.data_array

#plot_blts(viscpu_sim)
#plot_blts(healvis_sim)

compare_blts(viscpu_sim, healvis_sim)

In [9]:
### Using Cross-Correlations of pair (0, 1)
def print_across_freq(simulator):
    freqs = unique(simulator.uvdata.freq_array)
    fig, ax = plt.subplots(len(freqs), 1, sharex=True, 
                           subplot_kw={"ylabel":"|Visibility| [Jy]"})
    for i in range(len(freqs)):
        f = freqs[i]
        indices = simulator.uvdata.antpair2ind(0,1)
        ax[i].plot(np.real(simulator.uvdata.data_array[indices, 0, i, 0]), label="freq {} real".format(i))
        ax[i].plot(np.imag(simulator.uvdata.data_array[indices, 0, i, 0]), label="freq {} imag".format(i), linestyle="--")
            
    ax[0].legend()
    ax[-1].legend()
    ax[0].set_title("Cross-Correlations")
    ax[-1].set_xlabel("Observation Number")

                   
plot_blts(viscpu_sim)
plot_blts(healvis_sim)
plt.figure()
print_across_freq(viscpu_sim)
print_across_freq(healvis_sim)


NameError: name 'viscpu_sim' is not defined

In [3]:
a = [ [["time 1 ant 11", "time 1 ant12"], ["time 1 ant 21", "time 1 ant22"]], [["time 2 ant 11", "time 2 ant12"], ["time 2 ant 21", "time 2 ant22"]], [["time 3 ant 11", "time 3 ant12"], ["time 3 ant 21", "time 3 ant22"]] ] 
a= np.array(a)

b =  a.flatten()
print b[1]

SyntaxError: Missing parentheses in call to 'print'. Did you mean print(b[1])? (<ipython-input-3-2b197e99e6db>, line 5)

### 2 Telescope 2 Point Sources

In [ ]:
uvdata = myuvdata(ntel=2)
uvdata1 = myuvdata(ntel=2)
freqs = unique(uvdata.freq_array)
point_source_pos = np.array([[0, uvdata.telescope_lat_lon_alt[0] - np.pi/4], [0, uvdata.telescope_lat_lon_alt[0] + np.pi/4]])
point_source_flux = np.array([[1.0, 1.0]] * len(freqs))

viscpu_sim = VisCPU(
    uvdata=uvdata,
    sky_freqs=freqs,
    point_source_pos=point_source_pos,
    point_source_flux=point_source_flux,
    nside=2**4,
    real_dtype=np.float64,
    complex_dtype=np.complex128, 
)

healvis_sim = HealVis(
    uvdata=uvdata1,
    sky_freqs=freqs,
    point_source_pos=point_source_pos,
    point_source_flux=point_source_flux,
    nside=2 ** 4
)

viscpu = viscpu_sim.simulate()
healvis = healvis_sim.simulate()

print "MAX ABSOLUTE DIFF", np.max(np.abs(viscpu-healvis))

plot_blts(viscpu_sim)
plot_blts(healvis_sim)
plt.figure(figsize=(50,10))
plt.plot(viscpu.flatten() - healvis.flatten())

### Point Sources

In [ ]:
# put a point source in
point_source_pos = np.array([[0, uvdata2().telescope_lat_lon_alt[0] + np.pi/4]])
point_source_flux = np.array([[1.0]] * len(freqs))

viscpu_sim = VisCPU(
    uvdata=uvdata2(),
    sky_freqs=freqs,
    sky_intensity=uniform_sky(6),
    nside=2**4,
    real_dtype=np.float64,
    complex_dtype=np.complex128, 
)

healvis_sim = HealVis(
    uvdata=uvdata2(),
    sky_freqs=freqs,
    sky_intensity=uniform_sky(6),
    nside=2 ** 4
)

viscpu = viscpu_sim.simulate()
healvis6 = healvis_sim.simulate()

In [ ]:
np.max(np.abs(healvis6-heal_copy))

In [ ]:
plot_blts(viscpu_sim)
plot_blts(healvis_sim)

## PRISim

In [ ]:
simulator = PRISim(
    uvdata = myuvdata(),
    sky_intensity = uniform_sky(5),
    sky_freqs = freqs,
)

In [ ]:
vis = simulator.simulate()
plot_blts(simulator)

In [ ]:
uv = myuvdata()

In [ ]:
uv.tel